
### The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2020 Semester 1

## Assignment 1: Naive Bayes Classifiers

###### Submission deadline: 7 pm, Monday 20 Apr 2020

**Student Name(s):**    `PLEASE ENTER YOUR NAME(S) HERE`

**Student ID(s):**     `PLEASE ENTER YOUR ID(S) HERE`


This iPython notebook is a template which you will use for your Assignment 1 submission.

Marking will be applied on the four functions that are defined in this notebook, and to your responses to the questions at the end of this notebook (Submitted in a separate PDF file).

**NOTE: YOU SHOULD ADD YOUR RESULTS, DIAGRAMS AND IMAGES FROM YOUR OBSERVATIONS IN THIS FILE TO YOUR REPORT (the PDF file).**

You may change the prototypes of these functions, and you may write other functions, according to your requirements. We would appreciate it if the required functions were prominent/easy to find.

**Adding proper comments to your code is MANDATORY. **

In [278]:
import pandas as pd
import numpy as np
import math
from scipy import stats
from copy import copy, deepcopy

In [279]:
# read all datasets 
adult = pd.read_csv("datasets/adult.data", header = None)
bank = pd.read_csv("datasets/bank.data", header = None)
breast_cancer_wisconsin = pd.read_csv("datasets/breast-cancer-wisconsin.data", header = None)
car = pd.read_csv("datasets/car.data", header = None)
lymphography = pd.read_csv("datasets/lymphography.data", header = None)
mushroom = pd.read_csv("datasets/mushroom.data", header = None)
nursery = pd.read_csv("datasets/nursery.data", header = None)
somerville = pd.read_csv("datasets/somerville.data", header = None)
university = pd.read_csv("datasets/university.data", header = None)
wdbc = pd.read_csv("datasets/wdbc.data", header = None)
wine = pd.read_csv("datasets/wine.data", header = None)

In [280]:
def delete_missing_value(raw_dataset):
    rows = set(raw_dataset[raw_dataset.values == "?"].index)
    data = raw_dataset.drop(index = rows)
    return data

In [281]:
# This function should prepare the data by reading it from a file and converting it into a useful format for training and testing

def preprocess(df):
#     train=df.sample(frac=0.8,random_state=200)
#     test=df.drop(train.index)
    train = delete_missing_value(df)
    return train
train_data = preprocess(adult)


In [282]:
feature_number = 14
class_column = 14
EPSILON = 1/(2*train_data.shape[0])


In [283]:
#calculate prior
def find_nominal_prob(column):
    prior = {}
    total_number = len(column)
    classes = column.value_counts()
    for key in classes.keys():
        prior[key] = math.log2(classes[key]/total_number)
    return prior


In [302]:
def find_nominal_llh(attributes, llh_dict, data, feature_row):
    all_feature = data[feature_row].value_counts()
    for attribute in attributes:
        feature_dict = {feature_row:{}}
        subset = data[data[class_column] == attribute][feature_row]
        feature_dict[feature_row] = find_nominal_prob(subset)
        for check_feature in all_feature.keys():
            if check_feature not in feature_dict[feature_row].keys():
                feature_dict[feature_row][check_feature] = np.log2(EPSILON)
        if(llh_dict[attribute]):
            llh_dict[attribute].update(feature_dict)
        else:
            llh_dict[attribute] = feature_dict
            
    return llh_dict

llh_dict = dict.fromkeys(attributes)
find_nominal_llh(attributes, llh_dict, train_data, 3)


{'<=50K': {3: {'HS-grad': -1.4620290743606676,
   'Some-college': -2.0843139272724893,
   'Bachelors': -2.956714019989708,
   '11th': -4.517651477148488,
   'Assoc-voc': -4.55608620006414,
   '10th': -4.895725544404053,
   'Assoc-acdm': -4.912889336229962,
   'Masters': -4.997836370668083,
   '7th-8th': -5.439572191337715,
   '9th': -5.719285338318139,
   '12th': -6.024534692058871,
   '5th-6th': -6.3589537311294295,
   '1st-4th': -7.287569097892664,
   'Prof-school': -7.38001534665726,
   'Doctorate': -7.897622579576651,
   'Preschool': -8.975625091577925}},
 '>50K': {3: {'Bachelors': -1.8202870532997695,
   'HS-grad': -2.2151089713572127,
   'Some-college': -2.490508642356822,
   'Masters': -3.0318625914170663,
   'Prof-school': -4.208877017645698,
   'Assoc-voc': -4.447948180128776,
   'Doctorate': -4.744929917885908,
   'Assoc-acdm': -4.874212934830874,
   '10th': -6.991569885469033,
   '11th': -6.991569885469033,
   '7th-8th': -7.744929917885908,
   '12th': -8.016231939703301,
   

In [303]:
def find_numerical_llh(attributes, llh_dict, data, feature):
    """
    save mean and Std as llh
    """
    for attribute in attributes:
        feature_dict = {feature:{}}
        subset = subset = data[data[class_column] == attribute][feature]
        mean = np.mean(subset)
        std = np.std(subset)
        feature_dict[feature]["mean"] = mean
        feature_dict[feature]["std"] = std
        if(llh_dict[attribute]):
            llh_dict[attribute].update(feature_dict)
        else:
            llh_dict[attribute] = feature_dict
    return llh_dict
find_numerical_llh(attributes, llh_dict, train_data, 0)

{'<=50K': {3: {'HS-grad': -1.4620290743606676,
   'Some-college': -2.0843139272724893,
   'Bachelors': -2.956714019989708,
   '11th': -4.517651477148488,
   'Assoc-voc': -4.55608620006414,
   '10th': -4.895725544404053,
   'Assoc-acdm': -4.912889336229962,
   'Masters': -4.997836370668083,
   '7th-8th': -5.439572191337715,
   '9th': -5.719285338318139,
   '12th': -6.024534692058871,
   '5th-6th': -6.3589537311294295,
   '1st-4th': -7.287569097892664,
   'Prof-school': -7.38001534665726,
   'Doctorate': -7.897622579576651,
   'Preschool': -8.975625091577925},
  0: {'mean': 36.60806038668668, 'std': 13.46433407388627}},
 '>50K': {3: {'Bachelors': -1.8202870532997695,
   'HS-grad': -2.2151089713572127,
   'Some-college': -2.490508642356822,
   'Masters': -3.0318625914170663,
   'Prof-school': -4.208877017645698,
   'Assoc-voc': -4.447948180128776,
   'Doctorate': -4.744929917885908,
   'Assoc-acdm': -4.874212934830874,
   '10th': -6.991569885469033,
   '11th': -6.991569885469033,
   '7th-

In [304]:
# This function should calculat prior probabilities and likelihoods from the training data and using
# them to build a naive Bayes model
feature_types = [2, 0, 2, 1, 1, 0, 0, 0, 0, 0, 2, 2, 2, 0]

def train(data):
    prior = find_nominal_prob(data[class_column])
    llh_dict = dict.fromkeys(attributes)
    for i in range(feature_number):
        if(feature_types[i] == 2):
            find_numerical_llh(attributes, llh_dict, data, i)
        else:
            find_nominal_llh(attributes, llh_dict, data, i)
            
    return prior, llh_dict
prior, llh_dict = train(train_data)


In [307]:
# This function should predict classes for new items in a test dataset (for the purposes of this assignment, you
# can re-use the training data as a test set)

def predict(instance, prior, llh_dict):
    all_prob = {}
    for attribute in prior.keys():
        all_prob[attribute] = prior[attribute]
        for i in range(feature_number):
            prob = 0
            if(feature_types[i] == 2):
                mean = llh_dict[attribute][i]["mean"]
                std = llh_dict[attribute][i]["std"]
                prob = stats.norm.pdf(x=instance[i], loc=mean, scale=std)
                if prob > 0.0:
                    prob = np.log2(prob)
                else:
                    prob = np.log2(1 * 10 ** -8)
            else:
                prob = llh_dict[attribute][i][instance[i]]
            all_prob[attribute] += prob
    max_prob = -10000
    max_key = ""
    
    for prob in all_prob.keys():
        if(all_prob[prob] > max_prob):
            max_prob = all_prob[prob]
            max_key = prob
    return max_key

predict(train_data.iloc[100], prior, llh_dict)

'<=50K'

In [315]:
# This function should evaliate the prediction performance by comparing your model’s class outputs to ground
# truth labels

def evaluate(data):
    predict_list = []
    correct = 0
    total = data.shape[0]
    for index, row in data.iterrows():
        if row[14] == predict(row, prior, llh_dict):
            correct += 1
#         result_list = [index,row[14],predict(row, prior, llh_dict)]
#         predict_list.append(result_list)
    correct_rate = correct/total
    return correct_rate
%time
evaluate(train_data)


CPU times: user 1e+03 ns, sys: 1 µs, total: 2 µs
Wall time: 4.77 µs


0.821928254094556

## Questions 


If you are in a group of 1, you will respond to question (1), and **one** other of your choosing (two responses in total).

If you are in a group of 2, you will respond to question (1) and question (2), and **two** others of your choosing (four responses in total). 

A response to a question should take about 100–250 words, and make reference to the data wherever possible.

#### NOTE: you may develope codes or functions in respond to the question, but your formal answer should be added to a separate file.

### Q1
Try discretising the numeric attributes in these datasets and treating them as discrete variables in the na¨ıve Bayes classifier. You can use a discretisation method of your choice and group the numeric values into any number of levels (but around 3 to 5 levels would probably be a good starting point). Does discretizing the variables improve classification performance, compared to the Gaussian na¨ıve Bayes approach? Why or why not?

### Q2
Implement a baseline model (e.g., random or 0R) and compare the performance of the na¨ıve Bayes classifier to this baseline on multiple datasets. Discuss why the baseline performance varies across datasets, and to what extent the na¨ıve Bayes classifier improves on the baseline performance.

### Q3
Since it’s difficult to model the probabilities of ordinal data, ordinal attributes are often treated as either nominal variables or numeric variables. Compare these strategies on the ordinal datasets provided. Deterimine which approach gives higher classification accuracy and discuss why.

### Q4
Evaluating the model on the same data that we use to train the model is considered to be a major mistake in Machine Learning. Implement a hold–out or cross–validation evaluation strategy (you should implement this yourself and do not simply call existing implementations from `scikit-learn`). How does your estimate of effectiveness change, compared to testing on the training data? Explain why. (The result might surprise you!)

### Q5
Implement one of the advanced smoothing regimes (add-k, Good-Turing). Does changing the smoothing regime (or indeed, not smoothing at all) affect the effectiveness of the na¨ıve Bayes classifier? Explain why, or why not.

### Q6
The Gaussian na¨ıve Bayes classifier assumes that numeric attributes come from a Gaussian distribution. Is this assumption always true for the numeric attributes in these datasets? Identify some cases where the Gaussian assumption is violated and describe any evidence (or lack thereof) that this has some effect on the NB classifier’s predictions.